In [1]:
### Data Ingestion Pipeline

In [2]:
### Document Structure 

import os
from pathlib import Path
import numpy as np
import chromadb
from chromadb.config import Settings
from chromadb.utils.batch_utils import create_batches
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
from langchain_community.document_loaders import  PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from openai import OpenAI
import torch
from pathlib import PosixPath
import re

In [3]:
def numerical_sort_key(path: PosixPath) -> int:
    """
    Custom sorting key to extract the numerical part of the filename.
    
    It finds the first sequence of digits in the path's string representation
    and converts it to an integer for accurate sorting.
    """
    # Convert the PosixPath to a string
    path_str = str(path)
    
    # Use regex to find one or more digits (\d+)
    match = re.search(r'\d+', path_str)
    
    if match:
        # Convert the found digits (e.g., "5") into an integer (5)
        return int(match.group(0))
    else:
        # Return a fallback value (e.g., 0) if no number is found, 
        # though it's unlikely with your current file names.
        return 0

In [4]:
## Read the files from a pdf file in a directory 

def process_all_pdfs(directoryName):

    all_docs = []
    pdf_dir = Path(directoryName)


    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    pdf_files = sorted(pdf_files, key=numerical_sort_key)

    print(f"Found {len(pdf_files)} PDF files to process")

    for file in pdf_files:
        print(f"Processing file {file.name} ")

        try:
            loader = PyPDFLoader(str(file))
            documents = loader.load()

            for doc in documents:
                doc.metadata['source_file'] = file.name
                doc.metadata['file_type'] = 'pdf'
                doc.metadata['book_title'] = "book_"+file.name.split(".")[0]
            
            all_docs.extend(documents)
            print(f"Loaded {len(documents)} pages")

        except Exception as e:
            print(f" Error: {e}")
    
    return all_docs


In [ ]:
all_docs  = process_all_pdfs(directoryName="./knowledge_base")
                                                        

Found 7 PDF files to process
Processing file 1.pdf 
Loaded 118 pages
Processing file 2.pdf 
Loaded 359 pages
Processing file 3.pdf 
Loaded 455 pages
Processing file 4.pdf 
Loaded 243 pages
Processing file 5.pdf 
Loaded 891 pages
Processing file 6.pdf 
Loaded 671 pages
Processing file 7.pdf 
Loaded 638 pages


In [6]:
### Text splittign and Chunking 

def split_documents(documents, chunk_size=300, chunk_overlap=50):

    """ Split documents to smaller chunks for better RAG performance"""

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len, separators=["\n\n", "\n", ".", "!", "?", " ", ""])
    return text_splitter.split_documents(documents)

    


In [7]:
chunks = split_documents(all_docs)

In [8]:
chunks[1000].metadata

{'producer': 'ProcessText Group',
 'creator': '',
 'creationdate': 'D:20080417232732',
 'author': '',
 'title': '',
 'subject': '',
 'keywords': '',
 'source': 'knowledge_base/1.pdf',
 'total_pages': 118,
 'page': 60,
 'page_label': '61',
 'source_file': '1.pdf',
 'file_type': 'pdf',
 'book_title': 'book_1'}

In [9]:
class Embeddings:

    def __init__(self, model_name:str = "all-MiniLM-L6-v2"):

        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):

        try:
            
            print(f"Loading Embedded Model: {self.model_name}")
            
            self.model = SentenceTransformer(self.model_name)

            print(f"Model Loaded SuccessFully. Embedding Dimenison: {self.model.get_sentence_embedding_dimension()}")
            

        except Exception as e:

            print(f"Error Printing model {self.model_name}: {e}")
            pass

    def generate_embeddings(self, texts:List[str]):

        if not self.model:
            raise ValueError("Model Not Loaded")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        return embeddings


In [10]:
embeddings = Embeddings()

Loading Embedded Model: all-MiniLM-L6-v2
Model Loaded SuccessFully. Embedding Dimenison: 384


In [11]:
class VectorStore:

    def __init__(self, collection_name:str="harry_potter_clean_v1", persist_directory:str = "./knowledge_base/vector_store"):

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self.initialize_store()

    def initialize_store(self):
        
        try:
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            self.collection =  self.client.get_or_create_collection(name=self.collection_name, metadata={"hnsw:space": "cosine"}) # <--- Add this line)
            print(f"VectorStore Initialized : {self.collection_name}")
            
        except Exception as e:
            print(f"Error in Vectore Store: {e}")
            raise ValueError("Vector Store Not Created")

        
    def add_documents(self, documents: List[Any], embeddings: np.ndarray):


        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []


        for i, (doc, embedding ) in enumerate (zip(documents, embeddings)):
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)


            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            documents_text.append(doc.page_content)

            embeddings_list.append(embedding.tolist())
        
        try:
            # self.collection.add(ids=ids, embeddings=embeddings_list, metadatas=metadatas, documents=documents_text)
            # print(f"Successfully added {len(documents)} documents to the vector store")
            # print(f"Total documents in collection : {self.collection.count()}")
            # pass
            batches = create_batches(
                api=self.collection._client,
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            
            )

            for batch_ids, batch_embeddings, batch_metadatas, batch_documents in batches:
                self.collection.add(
                    ids=batch_ids,
                    embeddings=batch_embeddings,
                    metadatas=batch_metadatas,
                    documents=batch_documents
                )



        except Exception as e:
            print(f"Couldnt add data to the Vector Store {e}")
            pass
        




In [12]:
texts = [doc.page_content for doc in chunks]
embeddings_texts = embeddings.generate_embeddings(texts)

Batches:   0%|          | 0/831 [00:00<?, ?it/s]

In [13]:
vector_store = VectorStore()

VectorStore Initialized : harry_potter_clean_v1


In [14]:
embeddings_texts

array([[-0.0078942 , -0.00971607,  0.05580289, ...,  0.084111  ,
        -0.07735459, -0.04348418],
       [-0.06369741, -0.01781824, -0.00157029, ...,  0.02127322,
         0.05857925, -0.0687658 ],
       [-0.03480296,  0.01225663, -0.01224808, ...,  0.06081391,
         0.08534583, -0.06727676],
       ...,
       [-0.04582788,  0.02903817,  0.03912584, ...,  0.04433487,
        -0.01902489,  0.03317221],
       [-0.00095898,  0.02377967,  0.01469692, ...,  0.02154875,
        -0.09594901,  0.0410526 ],
       [-0.05036232,  0.1451934 ,  0.03554253, ...,  0.0233979 ,
        -0.00728701,  0.0656971 ]], shape=(26592, 384), dtype=float32)

In [15]:
vector_store.add_documents(chunks,embeddings_texts)

In [48]:
class RAGRetriever:

    def __init__(self, vector_store:VectorStore,embedding_manager:Embeddings):

        self.vector_store:VectorStore = vector_store
        self.embedding_manager:Embeddings = embedding_manager
        self.reranker_model = CrossEncoder('BAAI/bge-reranker-large', max_length=512)

        pass

    def retrive(self,query: str, top_k:int = 10, score_threshold:float = 0.0, filter: Dict[str,str]=None) -> List[Dict[str, Any]]:
        

        query_embedding = self.embedding_manager.generate_embeddings([query])[0]

        try:

            results = self.vector_store.collection.query(query_embeddings=[query_embedding.tolist()], 
                            n_results=top_k)

            retrieved_docs = []

            if results['documents'] and results['documents'][0]:
                documents =  results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids=results['ids'][0]

                for i, (doc_id, document, metadata,distance) in enumerate(zip(ids, documents, metadatas, distances)):

                    similarity_score = 1 - distance 

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id':doc_id,'content':document, 'metadata':metadata, 'similarity_score':similarity_score, 'distance':distance, 'rank':i+1
                        })

                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No Documents found")
            return retrieved_docs
        
        except Exception as e:
            print("No documents found")
            return []
            pass

        
        pass

    # 2. Add 'self' and proper type hints
    def rerank(self, query: str, documents: List[Dict[str, Any]], top_k: int = 5) -> List[Dict[str, Any]]:
        """
        Reranks retrieved documents using a cross-encoder model.
        """
        
        # Prepare sentence pairs for cross-encoder: [[query, doc1_content], [query, doc2_content], ...]
        sentence_pairs = [[query, doc['content']] for doc in documents]
        
        # Get relevance scores using the initialized model
        with torch.no_grad():
            scores = self.reranker_model.predict(sentence_pairs)
    
        # CRITICAL FIX: Save the score into the document dictionary
        for i, doc in enumerate(documents):
            doc['rerank_score'] = float(scores[i])

        # Sort by the new 'rerank_score' key
        ranked_docs = sorted(documents, key=lambda x: x['rerank_score'], reverse=True)
        
        return ranked_docs[:top_k]

In [49]:
rag_retriever = RAGRetriever(vector_store=vector_store, embedding_manager=embeddings)

In [50]:
rd= rag_retriever.retrive("dumbledore")

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)


In [43]:
rag_retriever.rerank("dumbledore",rd)

[]

In [20]:
llm_client = OpenAI(base_url="http://localhost:11434/v1", api_key="gemma:7b")

In [21]:
response = llm_client.chat.completions.create(model='gemma:7b', messages=[{"role":"system", "content":"hi i am tanveeer and you are ?"}], max_tokens=200, n=1, temperature=0.1)

In [22]:
response

ChatCompletion(id='chatcmpl-765', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='I am a large language model, trained by Google. I am here to assist you with information, answer your questions, and engage in meaningful conversations.', refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1763620323, model='gemma:7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tokens=31, prompt_tokens=33, total_tokens=64, completion_tokens_details=None, prompt_tokens_details=None))

In [51]:
def create_mock_response(content_string):
    # This defines the structure: response.choices[0].message.content
    MockMessage = type('MockMessage', (object,), {'content': content_string})
    MockChoice = type('MockChoice', (object,), {'message': MockMessage()})
    MockResponse = type('MockResponse', (object,), {'choices': [MockChoice()]})
    return MockResponse()

In [86]:
def rag_simple(query, retriever, llm, top_k=5, use_rerank=True):
    print(f"🔎 Searching for: {query}")
    
    # 1. RETRIEVE: Get 2x candidates to give the Reranker options to choose from
    initial_results = retriever.retrive(query, top_k=top_k * 2)
    
    if not initial_results:
        return "I couldn't find any information on that in the books."

    # 2. RERANK: Filter down to the best top_k
    if use_rerank:
        print("🔄 Reranking results...")
        final_results = retriever.rerank(query, initial_results, top_k=top_k)
    else:
        final_results = initial_results[:top_k]

    # 3. GENERATE
    context = "\n\n".join([f"[Extract]: {doc['content']}" for doc in final_results])
    
    messages = [
        {"role": "system", "content": "You are an expert on the Harry Potter universe. Answer strictly based on the provided book excerpts."},
        {"role": "user", "content": f"""
    You are an expert on the Harry Potter books.
    Answer the user's question with a concise, single paragraph.
    
    RULES:
    1. Base your answer **STRICTLY AND ONLY** on the context provided below.
    2. If the context does not contain the answer, state: 'The provided excerpts do not contain the answer to this question.'
    
    CONTEXT:
    ---
    {context}
    ---
    
    QUESTION: {query}
    """}
    ]

    # 3. TEMPORARILY RETURN THE RAW CONTEXT (Comment out the real generation step)
    # debug_text = f"--- DEBUG: RETRIEVED CONTEXT ---\n{context}\n\n--- END CONTEXT ---"
    # return create_mock_response(debug_text)


    response = llm.chat.completions.create(
        model='gemma:7b', 
        messages=messages, 
        max_tokens=500, 
        temperature=0.1
    )
    
    return response

In [62]:
def generate_character_profile(character_name, retriever, llm):
    query = f"Who is {character_name}? What is their appearance, personality, and key role?"
    
    # Get broad context
    results = retriever.retrive(query, top_k=15) # High k to catch mentions across books
    if retriever.reranker_model:
        results = retriever.rerank(query, results, top_k=7)
        
    context = "\n".join([doc['content'] for doc in results])

    messages = [
        {"role": "system", "content": "You are a magical historian compiling files for the Ministry of Magic."},
        {"role": "user", "content": f"""
        Create a dossier for: {character_name} using ONLY the provided text.
        
        Format the output exactly like this:
        **Name:** {character_name}
        **Affiliation:** (e.g. Hogwarts House, Death Eater, Order of the Phoenix)
        **Physical Description:** (Hair, eyes, distinct features)
        **Key Personality Traits:**
        **Notable Magical Feats/Events:**
        
        CONTEXT:
        {context}
        """}
    ]
    
    return llm.chat.completions.create(model='gemma:7b', messages=messages, temperature=0.3)

In [63]:
def debug_retrieval(query, retriever, top_k=5):
    print(f"\n🔎 --- DEBUGGING QUERY: '{query}' ---")
    
    # 1. Get Initial Vector Search Results
    initial_docs = retriever.retrive(query, top_k=top_k*2)
    print(f"📚 Found {len(initial_docs)} docs via Vector Search.")
    
    # 2. Apply Reranking
    reranked_docs = retriever.rerank(query, initial_docs, top_k=top_k)
    
    print(f"🥇 Top {top_k} Documents after Reranking:\n")
    
    for i, doc in enumerate(reranked_docs):
        print(f"Rank {i+1} | Score: {doc.get('rerank_score', 0):.4f}")
        print(f"Source: {doc['metadata'].get('source_file', 'Unknown')}")
        print(f"Content Snippet: {doc['content'][:150]}...") # Show first 150 chars
        print("-" * 50)

# --- TEST CASES ---
# Specific Fact
debug_retrieval("What is the password to the prefects' bathroom?", rag_retriever)

# Abstract Concept
debug_retrieval("How do Horcruxes work?", rag_retriever)


🔎 --- DEBUGGING QUERY: 'What is the password to the prefects' bathroom?' ---


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
📚 Found 10 docs via Vector Search.
🥇 Top 5 Documents after Reranking:

Rank 1 | Score: 0.0006
Source: 4.pdf
Content Snippet: Invisibility Cloak, looking around.
His immediate reaction was that it would be worth becoming a prefect just to be able to use this 
bathroom.  It wa...
--------------------------------------------------
Rank 2 | Score: 0.0004
Source: 2.pdf
Content Snippet: CHAPTER  FIVE 
 84  
They didn’t know the new year’s password, not having met a 
Gryffindor prefect yet, but help came almost immediately; they 
hea...
--------------------------------------------------
Rank 3 | Score: 0.0003
Source: 4.pdf
Content Snippet: "Password?" she said as they approached.
"Balderdash," said George, "a prefect downstairs told me."
 The portrait swung forward to reveal a hole in th...
--------------------------------------------------
Rank 4 | Score: 0.0001
Source: 1.pdf
Content Snippet: "Password?" she said. "Caput Draconis," said Percy, and t

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
📚 Found 10 docs via Vector Search.
🥇 Top 5 Documents after Reranking:

Rank 1 | Score: 0.7473
Source: 6.pdf
Content Snippet: HORCRUXES 
 501  
“I still don’t understand,” said Harry. 
“Well, it worked as a Horcrux is supposed to work — in other 
words, the fragment of soul...
--------------------------------------------------
Rank 2 | Score: 0.1315
Source: 6.pdf
Content Snippet: the term. A Horcrux is the word us ed for an object in which a per-
son has concealed part  of their soul.” 
“I don’t quite understand how th at works...
--------------------------------------------------
Rank 3 | Score: 0.0495
Source: 6.pdf
Content Snippet: about that precious fragment of his soul concealed within it. The 
point of a Horcrux is, as Professor Slughorn explained, to keep part 
of the self h...
--------------------------------------------------
Rank 4 | Score: 0.0009
Source: 7.pdf
Content Snippet: how to make a Horcrux but the time he asked Slughorn abou

In [117]:
answer = rag_simple("describe Harry's Sorting Hat experience", rag_retriever, llm_client,use_rerank=True)

🔎 Searching for: describe Harry's Sorting Hat experience


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
🔄 Reranking results...


In [118]:
answer

ChatCompletion(id='chatcmpl-145', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Harry's Sorting Hat experience was fraught with initial fear and uncertainty, but ultimately ended with him being sorted into Gryffindor. He recalled feeling terrified while waiting for the test, but the Hat itself seemed to acknowledge his anxieties, remarking on the increased branchings in that year's students. Despite the initial tension, Harry felt calmer in Dumbledore's office and was able to share his dream with the headmaster. The Hat revealed that it considers individual choices when making its decisions, as evidenced by its sorting of Snape into Slytherin despite his preference for Gryffindor.", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=None))], created=1763621841, model='gemma:7b', object='chat.completion', service_tier=None, system_fingerprint='fp_ollama', usage=CompletionUsage(completion_tok

In [119]:
answer.choices[0].message.content.split("\n")

["Harry's Sorting Hat experience was fraught with initial fear and uncertainty, but ultimately ended with him being sorted into Gryffindor. He recalled feeling terrified while waiting for the test, but the Hat itself seemed to acknowledge his anxieties, remarking on the increased branchings in that year's students. Despite the initial tension, Harry felt calmer in Dumbledore's office and was able to share his dream with the headmaster. The Hat revealed that it considers individual choices when making its decisions, as evidenced by its sorting of Snape into Slytherin despite his preference for Gryffindor."]

In [30]:
def evaluate_pipeline(test_dataset, retriever):
    total = len(test_dataset)
    hits = 0
    
    print(f"🧪 Starting Evaluation on {total} questions...\n")
    
    for item in test_dataset:
        query = item['question']
        target_keyword = item['expected_keyword'] # A word that MUST be in the retrieved text
        
        # Get results
        results = retriever.retrive(query, top_k=10)
        reranked = retriever.rerank(query, results, top_k=5)
        
        # Check if any of the top 5 docs contain the keyword
        found = False
        for doc in reranked:
            if target_keyword.lower() in doc['content'].lower():
                found = True
                break
        
        if found:
            hits += 1
            print(f"✅ PASS: {query}")
        else:
            print(f"❌ FAIL: {query} (Expected keyword: '{target_keyword}' not found)")

    print(f"\n📊 FINAL SCORE: {hits}/{total} ({(hits/total)*100:.1f}%) Hit Rate")

# Define a mini Golden Dataset
test_data = [
    {"question": "Who is the Half-Blood Prince?", "expected_keyword": "Snape"},
    {"question": "What position does Harry play in Quidditch?", "expected_keyword": "Seeker"},
    {"question": "Who killed Dumbledore?", "expected_keyword": "Snape"}, 
    {"question": "What is the name of Hagrid's dog?", "expected_keyword": "Fang"}
]

# Run the eval
evaluate_pipeline(test_data, rag_retriever)

🧪 Starting Evaluation on 4 questions...



Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
❌ FAIL: Who is the Half-Blood Prince? (Expected keyword: 'Snape' not found)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
✅ PASS: What position does Harry play in Quidditch?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
✅ PASS: Who killed Dumbledore?


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieved 10 documents (after filtering)
✅ PASS: What is the name of Hagrid's dog?

📊 FINAL SCORE: 3/4 (75.0%) Hit Rate
